In [1]:
#Import relevant libraries
import astropy
import numpy as np
import ccdproc
import os
import glob
import matplotlib.pyplot as plt
from astropy.nddata import CCDData
import scipy

In [2]:
#TASK 14 AND 15
#This is the other half of master science B, again we are subtracing the master dark and master bias from this and dividing by masterflat


masterflatB = astropy.nddata.CCDData.read('C:/Users/steve/OneDrive - University of Birmingham/LH Observatory Laboratory/Glob Cluter Final Code/GlobClusters/expflatB.fits', unit='adu')
masterbias = astropy.nddata.CCDData.read('C:/Users/steve/OneDrive - University of Birmingham/LH Observatory Laboratory/Glob Cluter Final Code/GlobClusters/expbias.fits', unit='adu')
masterdark = astropy.nddata.CCDData.read('C:/Users/steve/OneDrive - University of Birmingham/LH Observatory Laboratory/Glob Cluter Final Code/GlobClusters/expdark.fits', unit='adu')

#Subtract master bias and master dark from the science frames
#file_location = os.path.join('20191111/', 'science_B/', '*.fits')
#file_location = os.path.join('GlobularClusters/', 'science_B/', '*.fits')
filenames = glob.glob('C:/Users/steve/OneDrive - University of Birmingham/LH Observatory Laboratory/Glob Cluter Final Code/GlobClusters/20191111/science_B2/*.fits')

scienceimagesB = []

for file in filenames:
    scienceImg = astropy.nddata.CCDData.read(file, unit='adu')
    hdul = astropy.nddata.CCDData.to_hdu(scienceImg)
    scienceSub1 = scienceImg.subtract(masterbias)
    scienceSub2 = scienceSub1.subtract(masterdark)
    divscience = scienceSub2.divide(masterflatB)
    #divscience2 = divscience.divide(60) #to ensure exposure time is 1 sec
    scienceimagesB.append(divscience)
    
    
print(scienceimagesB)

INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


[CCDData([[ 154.72381731,  159.8825055 ,  114.80563672, ...,  160.10064207,
           167.82035633, -196.16067005],
         [ 168.4202557 ,  163.13011779,  175.12406154, ...,  172.97576847,
           197.87941227,  -92.00894832],
         [ 170.18146027,  175.72159477,  180.60981189, ...,  142.30795905,
           147.3368542 , -177.9041692 ],
         ...,
         [ 191.27526017,  158.47499904,  139.65971639, ...,  152.81213758,
           149.63667459, -390.98377936],
         [ 140.69785267,  193.05289495,  169.74571781, ...,  152.67404147,
           176.30649576,  -30.10286169],
         [ 154.44653218,  152.97450806,  173.21602776, ...,  168.47599794,
           140.8175263 ,  -56.6843552 ]], unit=''), CCDData([[ 161.64121425,  194.87242408,  153.71555456, ...,  152.2291443 ,
           173.69162244, -100.80478878],
         [ 153.28621475,  130.27653697,  179.15486608, ...,  159.89658698,
           169.6704792 ,  -73.06592955],
         [ 138.97793101,  158.72088764,  135.7

In [3]:
from skimage import data, draw
from skimage.registration import phase_cross_correlation
from scipy import ndimage

#Here we are aligning the second half of master science B

scienceimage0 = astropy.nddata.CCDData.read('C:/Users/steve/OneDrive - University of Birmingham/LH Observatory Laboratory/Glob Cluter Final Code/GlobClusters/20191111/science_B/20191111.00000030.fits', unit='adu')

scienceimagesBcorr = []
for i in range(len(scienceimagesB)):
    detected_shiftB = phase_cross_correlation(np.array(scienceimage0), np.array(scienceimagesB[i]), upsample_factor = 1000)
    print(detected_shiftB[0])
    scienceimagesB1 = scipy.ndimage.shift(scienceimagesB[i], detected_shiftB[0])
    scienceimagesB11 = astropy.nddata.CCDData(scienceimagesB1, unit = 'adu')
    scienceimagesBcorr.append(scienceimagesB11)

[-2.957  3.112]
[-3.156  2.781]
[-2.91   1.783]
[-3.674  4.073]
[-3.193  1.582]
[-3.286  4.116]
[-3.843  3.405]
[-3.853  2.973]
[-4.246  5.443]
[-4.194  4.419]
[-4.968  6.8  ]
[-4.946  5.84 ]
[-4.873  7.204]


In [4]:
scienceimagesBcorr = astropy.nddata.CCDData(scienceimagesBcorr, unit='adu')
print(scienceimagesBcorr)

[[[  0.           0.           0.         ... 142.90822943 178.38623518
   180.54702178]
  [  0.           0.           0.         ... 179.13483851 163.42405746
   164.96763096]
  [  0.           0.           0.         ... 175.15562129 170.11149067
   162.03816758]
  ...
  [  0.           0.           0.         ...   0.           0.
     0.        ]
  [  0.           0.           0.         ...   0.           0.
     0.        ]
  [  0.           0.           0.         ...   0.           0.
     0.        ]]

 [[  0.           0.           0.         ... 137.50560691 192.70177947
   151.17237568]
  [  0.           0.           0.         ... 182.1735537  187.64491398
   154.90469644]
  [  0.           0.           0.         ... 157.24011561 172.45297029
   165.22771266]
  ...
  [  0.           0.           0.         ...   0.           0.
     0.        ]
  [  0.           0.           0.         ...   0.           0.
     0.        ]
  [  0.           0.           0.         ...  

In [5]:
print((scienceimagesBcorr.shape))

(13, 4096, 4096)


In [6]:
#This is the combines alignment of master science B for the second half

ScienceB = ccdproc.combine((scienceimagesBcorr), method = 'median', format = 'fits')
hdu1 = CCDData.to_hdu(ScienceB)
hdu1.writeto('MasterScienceB2.fits', overwrite = True)